In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import h5py
import numpy as np
import labrotation.file_handling as fh
import os
import pandas as pd
import labrotation.two_photon_session as tps
import datadoc_util
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from datetime import datetime as dt

In [ ]:
colors = mcolors.TABLEAU_COLORS  # has 10 unique colors

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict.keys():
    data_docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    data_docu_folder = fh.open_dir("Open Data Documentation folder")
print(data_docu_folder)

In [ ]:
ddoc = datadoc_util.DataDocumentation(data_docu_folder)
ddoc.loadDataDoc()

# Get seizure traces
The h5 file should contain groups named event uuids (i.e. each seizure has its own uuid). Inside the group, all data sources (labview time, labview speed, nikon mean fluo, lfp (if present) t and lfp values etc.) should have an own dataset.

In [ ]:
events_list_fpath = os.path.join(env_dict["DATA_DOCU_FOLDER"], "events_list.xlsx")
assert os.path.exists(events_list_fpath)

In [ ]:
df_events = pd.read_excel(events_list_fpath)
df_events = df_events[df_events["event_type"] == "sz"] 

In [ ]:
event_traces_fpath = fh.open_file("Open .h5 file containing assembled traces for all seizures!")
print(event_traces_fpath)

In [ ]:
traces_dict = dict()  
traces_meta_dict = dict()
# first keys are event uuids, inside the following dataset names:
# 'lfp_mov_t', 'lfp_mov_y', 'lfp_t', 'lfp_y', 'lv_dist', 'lv_rounds', 
# 'lv_running', 'lv_speed', 'lv_t_s', 'lv_totdist', 'mean_fluo'
with h5py.File(event_traces_fpath, "r") as hf:
    for uuid in hf.keys():
        session_dataset_dict = dict() 
        session_meta_dict = dict()
        for dataset_name in hf[uuid].keys():
            session_dataset_dict[dataset_name] = np.array(hf[uuid][dataset_name])
        for attr_name in hf[uuid].attrs:
            session_meta_dict[attr_name] = hf[uuid].attrs[attr_name]
        traces_dict[uuid] = session_dataset_dict.copy()
        traces_meta_dict[uuid] = session_meta_dict.copy()

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.gca(projection='3d')

zs = [i*1.0 for i in range(len(traces_dict.keys()))]
verts = []
for uuid in traces_dict.keys():
    xs = traces_dict[uuid]["lv_t_s"]
    ys = (traces_dict[uuid]["mean_fluo"] - min(traces_dict[uuid]["mean_fluo"]))/max(traces_dict[uuid]["mean_fluo"])
    verts.append(list(zip(xs, ys)))

poly = PolyCollection(verts)
poly.set_alpha(0.4)
ax.add_collection3d(poly, zs=zs, zdir='y')


ax.set_xlabel('X')
ax.set_xlim3d(0, 800)
ax.set_ylabel('Y')
ax.set_ylim3d(0, 25)
ax.set_zlabel('Z')
#ax.set_zlim3d(0, 500)

plt.show()

In [ ]:
for uuid in traces_dict.keys():
    print(f"{len(traces_dict[uuid]['lv_t_s'])} - {len(traces_dict[uuid]['mean_fluo'])} = {len(traces_dict[uuid]['lv_t_s']) - len(traces_dict[uuid]['mean_fluo'])}")

In [ ]:
for uuid in traces_dict.keys():
    print(traces_dict[uuid]["lv_t_s"][0])

In [ ]:
df_events[df_events["event_uuid"] == "f0442bebcd1a4291a8d0559eb47df08e"]

In [ ]:
def get_datetime_for_fname():
    now = dt.now()
    return f"{now.year:04d}{now.month:02d}{now.day:02d}-{now.hour:02d}{now.minute:02d}{now.second:02d}"

# Plot results

In [ ]:
save_figs = True
file_format = ".jpg"  # ".eps"

In [ ]:
AMPLITUDE = 100.0
offset = 0
fig = plt.figure(figsize=(18,18))
color_index = 0
mouse_ids = df_events.mouse_id.unique()
prev_range = 0.0
for (mouse_id, event_uuid), g in df_events.sort_values(["window_type", "sz_index"]).groupby(["mouse_id", "event_uuid"]):
    print(event_uuid)
    color = colors[list(colors.keys())[df_events.mouse_id.unique().searchsorted(mouse_id)]]
    plt.scatter(traces_dict[event_uuid]["lv_t_s"], AMPLITUDE*(traces_dict[event_uuid]["mean_fluo"] - min(traces_dict[event_uuid]["mean_fluo"]))/(max(traces_dict[event_uuid]["mean_fluo"]) - min(traces_dict[event_uuid]["mean_fluo"]))+offset, color=color, s=2)
    #plt.scatter(traces_dict[event_uuid]["lv_t_s"], traces_dict[event_uuid]["mean_fluo"] + prev_range + offset, color=color, s=2)
    # plot recording end points to show where stitching happened
    plt.vlines(traces_dict[event_uuid]["lv_t_s"][traces_meta_dict[event_uuid]["recording_break_points"]], offset, offset+AMPLITUDE, color="black")
    # plot segment borders (bl, sz, am)
    #plt.vlines(traces_dict[event_uuid]["lv_t_s"][traces_meta_dict[event_uuid]["segment_type_break_points"]], offset, offset+AMPLITUDE, color="black")
    prev_range += max(traces_dict[event_uuid]["mean_fluo"]) - min(traces_dict[event_uuid]["mean_fluo"])
    offset+=1.1*AMPLITUDE
#plt.axis("off")
plt.yticks([])
plt.xlabel("Time (s)", fontsize=14)
plt.xlim((250, 500))  # 250, 500
if save_figs:
    out_fpath = f"D:\\Downloads\\traces_{get_datetime_for_fname()}{file_format}"
    plt.savefig(out_fpath,bbox_inches='tight', dpi=300)
    print(f"Saved as {out_fpath}")
plt.show()


In [ ]:
fig = plt.figure(figsize=(18,18))
color_index = 0
mouse_ids = df_events.mouse_id.unique()
offset = 0.0
for (mouse_id, event_uuid), g in df_events.sort_values(["window_type", "sz_index"]).groupby(["mouse_id", "event_uuid"]):
    print(event_uuid)
    color = colors[list(colors.keys())[df_events.mouse_id.unique().searchsorted(mouse_id)]]
    plt.scatter(traces_dict[event_uuid]["lfp_t"], traces_dict[event_uuid]["lfp_y"]+offset, color=color, s=0.1)
    #plt.vlines(traces_dict[event_uuid]["lfp_t"][traces_meta_dict[event_uuid]["recording_break_points_lfp"]], offset-0.1, offset+0.4, color="black")
    plt.vlines(traces_dict[event_uuid]["lfp_t"][traces_meta_dict[event_uuid]["segment_type_break_points_lfp"]], offset-0.1, offset+0.4, color="grey")
    offset+=0.5
#plt.axis("off")
plt.yticks([])
plt.xlabel("Time (s)", fontsize=14)
plt.xlim((300, 480))
if save_figs:
    out_fpath = f"D:\\Downloads\\tmev_sz_traces_{get_datetime_for_fname()}{file_format}"
    plt.savefig(out_fpath,bbox_inches='tight', dpi=300)
    print(f"Saved as {out_fpath}")
plt.show()

# Combine all data sources

In [ ]:
AMPLITUDE = 100.0

fig = plt.figure(figsize=(24,48))
color_index = 0
mouse_ids = df_events.mouse_id.unique()
for (mouse_id, event_uuid), g in df_events.sort_values(["window_type", "sz_index"]).groupby(["mouse_id", "event_uuid"]):
    print(event_uuid)
    color = colors[list(colors.keys())[df_events.mouse_id.unique().searchsorted(mouse_id)]]
    
    # plot labview speed
    plt.scatter(traces_dict[event_uuid]["lv_t_s"], AMPLITUDE*(traces_dict[event_uuid]["lv_speed"] - min(traces_dict[event_uuid]["lv_speed"]))/(max(traces_dict[event_uuid]["lv_speed"]) - min(traces_dict[event_uuid]["lv_speed"]))+offset, color=color, s=2)
    offset+=AMPLITUDE + 0.1*AMPLITUDE
    
    # plot lfp movement
    plt.scatter(traces_dict[event_uuid]["lfp_mov_t"], AMPLITUDE*(traces_dict[event_uuid]["lfp_mov_y"] - min(traces_dict[event_uuid]["lfp_mov_y"]))/(max(traces_dict[event_uuid]["lfp_mov_y"]) - min(traces_dict[event_uuid]["lfp_mov_y"]))+offset, color=color, s=2)
    offset+=AMPLITUDE + 0.1*AMPLITUDE
    
    # plot lfp
    plt.scatter(traces_dict[event_uuid]["lfp_t"], AMPLITUDE*(traces_dict[event_uuid]["lfp_y"] - min(traces_dict[event_uuid]["lfp_y"]))/(max(traces_dict[event_uuid]["lfp_y"]) - min(traces_dict[event_uuid]["lfp_y"]))+offset, color=color, s=2)
    offset+=AMPLITUDE + 0.1*AMPLITUDE
    
    # plot nikon mean fluorescence
    plt.scatter(traces_dict[event_uuid]["lv_t_s"], AMPLITUDE*(traces_dict[event_uuid]["mean_fluo"] - min(traces_dict[event_uuid]["mean_fluo"]))/(max(traces_dict[event_uuid]["mean_fluo"]) - min(traces_dict[event_uuid]["mean_fluo"]))+offset, color=color, s=2)
    offset+=AMPLITUDE + 0.5*AMPLITUDE
    
#plt.axis("off")
plt.yticks([])
plt.xlabel("Time (s)", fontsize=14)

#plt.savefig(out_fpath,bbox_inches='tight', dpi=300)
if save_figs:
    out_fpath = f"D:\\Downloads\\all_traces_{get_datetime_for_fname()}{file_format}"
    plt.savefig(out_fpath,bbox_inches='tight', dpi=300)
    print(f"Saved as {out_fpath}")
plt.show()

# Plot locomotion and Nikon together

In [ ]:
AMPLITUDE = 100.0

fig = plt.figure(figsize=(18,36))
color_index = 0
mouse_ids = df_events.mouse_id.unique()
for (mouse_id, event_uuid), g in df_events.sort_values(["window_type", "sz_index"]).groupby(["mouse_id", "event_uuid"]):
    print(event_uuid)
    color = colors[list(colors.keys())[df_events.mouse_id.unique().searchsorted(mouse_id)]]
    # plot labview loco
    plt.scatter(traces_dict[event_uuid]["lv_t_s"], AMPLITUDE*(traces_dict[event_uuid]["lv_speed"] - min(traces_dict[event_uuid]["lv_speed"]))/(max(traces_dict[event_uuid]["lv_speed"]) - min(traces_dict[event_uuid]["lv_speed"]))+offset, color=color, s=2)
    
    plt.vlines(traces_dict[event_uuid]["lv_t_s"][traces_meta_dict[event_uuid]["recording_break_points"]], offset-0.05*AMPLITUDE, offset+2.1*AMPLITUDE, color="black")
    
    
    offset+=AMPLITUDE + 0.05*AMPLITUDE    
    
    # plot Nikon
    plt.scatter(traces_dict[event_uuid]["lv_t_s"], AMPLITUDE*(traces_dict[event_uuid]["mean_fluo"] - min(traces_dict[event_uuid]["mean_fluo"]))/(max(traces_dict[event_uuid]["mean_fluo"]) - min(traces_dict[event_uuid]["mean_fluo"]))+offset, color=color, s=2)
    
    
    
    
    offset+=2*AMPLITUDE
    
#plt.axis("off")
plt.yticks([])
plt.xlabel("Time (s)", fontsize=14)
#plt.xlim((250, 500))
if save_figs:
    out_fpath = f"D:\\Downloads\\traces_w_loco_{get_datetime_for_fname()}{file_format}"
    plt.savefig(out_fpath,bbox_inches='tight', dpi=300)
    print(f"Saved as {out_fpath}")
plt.show()


# Test accumulative data
Rounds and total distance should be continuous

In [ ]:
AMPLITUDE = 100.0
offset = 0
fig = plt.figure(figsize=(18,18))
color_index = 0
mouse_ids = df_events.mouse_id.unique()
prev_range = 0.0
for (mouse_id, event_uuid), g in df_events.sort_values(["window_type", "sz_index"]).groupby(["mouse_id", "event_uuid"]):
    print(event_uuid)
    color = colors[list(colors.keys())[df_events.mouse_id.unique().searchsorted(mouse_id)]]
    plt.scatter(traces_dict[event_uuid]["lv_t_s"], AMPLITUDE*(traces_dict[event_uuid]["lv_totdist_abs"] - min(traces_dict[event_uuid]["lv_totdist_abs"]))/(max(traces_dict[event_uuid]["lv_totdist_abs"]) - min(traces_dict[event_uuid]["lv_totdist_abs"]))+offset, color=color, s=2)
    #plt.scatter(traces_dict[event_uuid]["lv_t_s"], traces_dict[event_uuid]["mean_fluo"] + prev_range + offset, color=color, s=2)
    # plot recording end points to show where stitching happened
    plt.vlines(traces_dict[event_uuid]["lv_t_s"][traces_meta_dict[event_uuid]["break_points"]], offset, offset+AMPLITUDE, color="black")
    # plot segment borders (bl, sz, am)
    #plt.vlines(traces_dict[event_uuid]["lv_t_s"][traces_meta_dict[event_uuid]["segment_type_break_points"]], offset, offset+AMPLITUDE, color="black")
    prev_range += max(traces_dict[event_uuid]["mean_fluo"]) - min(traces_dict[event_uuid]["mean_fluo"])
    offset+=1.1*AMPLITUDE
#plt.axis("off")
plt.yticks([])
plt.xlabel("Time (s)", fontsize=14)
plt.xlim((290,500))
plt.show()

In [ ]:
AMPLITUDE = 100.0
offset = 0
fig = plt.figure(figsize=(18,18))
color_index = 0
mouse_ids = df_events.mouse_id.unique()
prev_range = 0.0
for (mouse_id, event_uuid), g in df_events.sort_values(["window_type", "sz_index"]).groupby(["mouse_id", "event_uuid"]):
    print(event_uuid)
    color = colors[list(colors.keys())[df_events.mouse_id.unique().searchsorted(mouse_id)]]
    plt.scatter(traces_dict[event_uuid]["lv_t_s"], AMPLITUDE*(traces_dict[event_uuid]["lv_speed"] - min(traces_dict[event_uuid]["lv_speed"]))/(max(traces_dict[event_uuid]["lv_speed"]) - min(traces_dict[event_uuid]["lv_speed"]))+offset, color=color, s=2)
    #plt.scatter(traces_dict[event_uuid]["lv_t_s"], traces_dict[event_uuid]["mean_fluo"] + prev_range + offset, color=color, s=2)
    # plot recording end points to show where stitching happened
    plt.vlines(traces_dict[event_uuid]["lv_t_s"][traces_meta_dict[event_uuid]["recording_break_points"]], offset, offset+AMPLITUDE, color="black")
    # plot segment borders (bl, sz, am)
    #plt.vlines(traces_dict[event_uuid]["lv_t_s"][traces_meta_dict[event_uuid]["segment_type_break_points"]], offset, offset+AMPLITUDE, color="black")
    prev_range += max(traces_dict[event_uuid]["mean_fluo"]) - min(traces_dict[event_uuid]["mean_fluo"])
    offset+=1.1*AMPLITUDE
#plt.axis("off")
plt.yticks([])
plt.xlabel("Time (s)", fontsize=14)
plt.xlim((290,500))
plt.show()

In [ ]:
AMPLITUDE = 100.0
offset = 0
fig = plt.figure(figsize=(18,18))
color_index = 0
mouse_ids = df_events.mouse_id.unique()
prev_range = 0.0
for (mouse_id, event_uuid), g in df_events.sort_values(["window_type", "sz_index"]).groupby(["mouse_id", "event_uuid"]):
    print(event_uuid)
    color = colors[list(colors.keys())[df_events.mouse_id.unique().searchsorted(mouse_id)]]
    plt.scatter(traces_dict[event_uuid]["lv_t_s"], AMPLITUDE*(traces_dict[event_uuid]["lv_rounds"] - min(traces_dict[event_uuid]["lv_rounds"]))/(max(traces_dict[event_uuid]["lv_rounds"]) - min(traces_dict[event_uuid]["lv_rounds"]))+offset, color=color, s=2)
    #plt.scatter(traces_dict[event_uuid]["lv_t_s"], traces_dict[event_uuid]["mean_fluo"] + prev_range + offset, color=color, s=2)
    # plot recording end points to show where stitching happened
    plt.vlines(traces_dict[event_uuid]["lv_t_s"][traces_meta_dict[event_uuid]["recording_break_points"]], offset, offset+AMPLITUDE, color="black")
    # plot segment borders (bl, sz, am)
    #plt.vlines(traces_dict[event_uuid]["lv_t_s"][traces_meta_dict[event_uuid]["segment_type_break_points"]], offset, offset+AMPLITUDE, color="black")
    prev_range += max(traces_dict[event_uuid]["mean_fluo"]) - min(traces_dict[event_uuid]["mean_fluo"])
    offset+=1.1*AMPLITUDE
#plt.axis("off")
plt.yticks([])
plt.xlabel("Time (s)", fontsize=14)
plt.xlim((250, 500))  # 250, 500
plt.show()